# Bathymetry Visualization and Analysis

This notebook creates advanced visualizations of the bathymetry results:
1. 3D bathymetry visualization
2. Depth profile analysis
3. Uncertainty visualization
4. Comparison with reference data

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
project_dir = Path().absolute().parent
if project_dir.name != 'sdb_project':
    project_dir = project_dir / 'sdb_project'
sys.path.append(str(project_dir))

# Import project modules
from src.visualize import create_3d_surface

## Load Configuration and Results

In [ ]:
# Load region configuration
config_path = project_dir / 'config' / 'location_config.json'
with open(config_path) as f:
    config = json.load(f)

region_slug = config['region_name'].lower().replace(' ', '_')

# Setup paths
output_dir = project_dir / 'outputs' / region_slug

# Load predictions and metadata
ensemble_pred = np.load(output_dir / 'ensemble_prediction.npy')
with open(output_dir / 'bathymetry_results.json') as f:
    results = json.load(f)

print(f"Loaded results for {config['region_name']}")
print(f"Depth range: {results['depth_statistics']['min_depth']:.1f}m to {results['depth_statistics']['max_depth']:.1f}m")

## Create 3D Bathymetry Surface

In [ ]:
# Create coordinate grids
lats = np.linspace(config['aoi']['min_lat'], config['aoi']['max_lat'], ensemble_pred.shape[0])
lons = np.linspace(config['aoi']['min_lon'], config['aoi']['max_lon'], ensemble_pred.shape[1])
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Create 3D surface plot
fig = go.Figure(data=[go.Surface(
    z=ensemble_pred,
    x=lon_grid,
    y=lat_grid,
    colorscale='Viridis',
    colorbar=dict(title='Depth (m)')
)])

fig.update_layout(
    title=f'3D Bathymetry - {config["region_name"]}',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Depth (m)',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    )
)

# Save 3D plot
fig.write_html(output_dir / 'bathymetry_3d.html')

# Display plot
fig.show()

## Depth Profile Analysis

In [ ]:
def extract_profile(data, start_idx, end_idx, axis=0):
    """Extract depth profile along specified axis"""
    if axis == 0:
        profile = data[:, start_idx]
        coords = lats
        coord_name = 'Latitude'
    else:
        profile = data[start_idx, :]
        coords = lons
        coord_name = 'Longitude'
    
    return profile, coords, coord_name

# Create depth profiles
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Longitudinal profile
center_lat_idx = ensemble_pred.shape[0] // 2
profile_lon, coords_lon, coord_name_lon = extract_profile(ensemble_pred, center_lat_idx, None, axis=1)

ax1.plot(coords_lon, profile_lon, 'b-', label='Bathymetry')
ax1.set_xlabel(coord_name_lon)
ax1.set_ylabel('Depth (m)')
ax1.set_title('Longitudinal Depth Profile')
ax1.grid(True)
ax1.invert_yaxis()

# Latitudinal profile
center_lon_idx = ensemble_pred.shape[1] // 2
profile_lat, coords_lat, coord_name_lat = extract_profile(ensemble_pred, center_lon_idx, None, axis=0)

ax2.plot(coords_lat, profile_lat, 'r-', label='Bathymetry')
ax2.set_xlabel(coord_name_lat)
ax2.set_ylabel('Depth (m)')
ax2.set_title('Latitudinal Depth Profile')
ax2.grid(True)
ax2.invert_yaxis()

plt.tight_layout()
plt.savefig(output_dir / 'depth_profiles.png')
plt.show()

## Load and Compare with GEBCO Reference

In [ ]:
# Load GEBCO reference data
import rasterio
gebco_path = project_dir / 'data' / 'gebco_reference' / 'extracted' / f'{region_slug}_bathymetry.tif'

with rasterio.open(gebco_path) as src:
    gebco_data = src.read(1)

# Create comparison plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# GEBCO plot
im1 = ax1.imshow(gebco_data, cmap='viridis')
ax1.set_title('GEBCO Reference')
plt.colorbar(im1, ax=ax1, label='Depth (m)')

# SDB plot
im2 = ax2.imshow(ensemble_pred, cmap='viridis')
ax2.set_title('SDB Prediction')
plt.colorbar(im2, ax=ax2, label='Depth (m)')

# Difference plot
difference = ensemble_pred - gebco_data
im3 = ax3.imshow(difference, cmap='RdYlBu')
ax3.set_title('Difference (SDB - GEBCO)')
plt.colorbar(im3, ax=ax3, label='Difference (m)')

plt.tight_layout()
plt.savefig(output_dir / 'reference_comparison.png')
plt.show()

# Calculate statistics
valid_mask = ~np.isnan(ensemble_pred) & ~np.isnan(gebco_data)
diff_stats = {
    'mean_difference': float(np.mean(difference[valid_mask])),
    'std_difference': float(np.std(difference[valid_mask])),
    'rmse': float(np.sqrt(np.mean(difference[valid_mask]**2))),
    'max_difference': float(np.max(np.abs(difference[valid_mask])))
}

print("\nComparison Statistics:")
for key, value in diff_stats.items():
    print(f"{key}: {value:.2f}m")

# Save statistics
with open(output_dir / 'comparison_statistics.json', 'w') as f:
    json.dump(diff_stats, f, indent=2)

## Export Results Summary

In [ ]:
# Create summary report
summary = {
    'region': config['region_name'],
    'aoi': config['aoi'],
    'bathymetry_statistics': results['depth_statistics'],
    'uncertainty_statistics': results['uncertainty_statistics'],
    'reference_comparison': diff_stats,
    'visualization_files': {
        '3d_view': str(output_dir / 'bathymetry_3d.html'),
        'depth_profiles': str(output_dir / 'depth_profiles.png'),
        'reference_comparison': str(output_dir / 'reference_comparison.png')
    }
}

# Save summary
with open(output_dir / 'visualization_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("Visualization results saved to:", output_dir)
print("Generated visualization files:")
for name, path in summary['visualization_files'].items():
    print(f"- {name}: {path}")